In [3]:
# nuclio: ignore
import nuclio

In [4]:
%run set_env.ipynb

In [5]:
import os
import sys
import time

In [6]:
def remove(path):
    try:
     if os.path.exists(path):
        os.remove(path)
    except OSError:
        print ("Unable to remove file: %s"%path)

In [7]:
def cleanup(context,number_of_hours, path):
    """
    Removes files from the passed in path that are older than or equal 
    to the number_of_days
    """
    time_in_secs = time.time() - (number_of_hours * 60)
    for root, dirs, files in os.walk(path, topdown=False):
        for file_ in files:
            full_path = os.path.join(root, file_)
            stat = os.stat(full_path)
            
            if stat.st_mtime <= time_in_secs:
                #context.logger.info("Removing %s"%full_path )
                remove(full_path)
            else:
                context.logger.info("Not Removing %s"%full_path )
            
        #if not os.listdir(root):
        #    remove(root)

In [8]:
def handler(context,event):
    cleanup(context,int(os.getenv('HOURS',1)), os.path.join(os.getenv('NETAPP_MOUNT_PATH'),os.getenv('FEATURES_TABLE') ) )
    return ''

In [10]:
# nuclio: ignore
spec = nuclio.ConfigSpec(config={"spec.triggers.inference.kind":"cron",
                                "spec.triggers.inference.attributes.interval" :"5m",
                                "spec.readinessTimeoutSeconds" : 360 },
                         env={'FEATURES_TABLE': FEATURES_TABLE,
                              'NETAPP_MOUNT_PATH' : NETAPP_MOUNT_PATH,
                              'HOURS' : 6
                             },
                        ).with_v3io().add_volume(IGZ_MOUNT_PATH,IGZ_CONTAINER_PATH,kind='v3io',name='v3io').add_volume(NETAPP_MOUNT_PATH,NETAPP_PVC_CLAIM,kind='pvc',name='netapp')

In [14]:
# nuclio: ignore
addr = nuclio.deploy_file(source=os.path.join(APP_DIR,'cleanup.ipynb'),
                                                  name='cleanup',project="netops",verbose=True, spec=spec, tag='v1.1')

[nuclio] 2020-03-19 13:48:29,376 updating cleanup
[nuclio] 2020-03-19 13:48:29,453 deploying ...
[nuclio] 2020-03-19 13:49:09,937 (info) Build complete
[nuclio] 2020-03-19 13:49:09,937 {'level': 'info', 'message': 'Build complete', 'name': 'deployer', 'result': {'Image': 'nuclio/netops-cleanup-processor:latest', 'UpdatedFunctionConfig': {'metadata': {'annotations': {'nuclio.io/generated_by': 'function generated at 19-03-2020 by admin from /User/netops-netapp/cleanup.ipynb'}, 'labels': {'nuclio.io/project-name': 'netops', 'nuclio.io/tag': 'v1.1'}, 'name': 'cleanup', 'namespace': 'default-tenant'}, 'spec': {'build': {'codeEntryType': 'sourceCode', 'functionSourceCode': 'IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5OdWNsaW9FeHBvcnRlciBvbiAyMDIwLTAzLTE5IDEzOjQ4CgppbXBvcnQgb3MKaW1wb3J0IHN5cwppbXBvcnQgdGltZQoKZGVmIHJlbW92ZShwYXRoKToKICAgIHRyeToKICAgICBpZiBvcy5wYXRoLmV4aXN0cyhwYXRoKToKICAgICAgICBvcy5yZW1vdmUocGF0aCkKICAgIGV4Y2VwdCBPU0Vycm9yOgogICAgICAgIHByaW50ICgiVW5hYmxlIHRvIHJlbW92ZSBmaWxlOiAlcyI